# Setting the ground

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset, WeightedRandomSampler
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
import requests
from random import randint, sample
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import f1_score
from torchvision.models import resnet50
from torchvision.models.resnet import ResNet50_Weights
import cv2
from collections import defaultdict
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import math
from matplotlib.ticker import FuncFormatter
import matplotlib as mpl
import matplotlib.patches as mpatches

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
drive.mount( '/content/drive', force_remount=True )
os.chdir("/content/drive/MyDrive/") #change the path so that it leads to your data

In [ ]:
#define this to sample Google Street View Images: 
api_key = 'some_string'

In [ ]:
!pip install timm #you need to install it every time in the cloud to work with DeiT models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00


# Collect the data

## Produce targets

I first download the data for neighbourhoods ("buurten") from the website of the Dutch Health Monitor (there is a link to Statline there, use that one, you don't need to submit a request, the data is openly available), only for variables Moderate to high risk of depression and anxiety and High risk of depression and anxiety. I used the results for adults and elderly from 2020. The following is done with this data. 

In [ ]:
rivm = pd.read_csv('./NL_gezondheidsdata/rivm_depri_buurt_dataportal.csv', delimiter=';')
display(rivm)

In [ ]:
rivm = rivm.drop(['Marges','Leeftijd', 'Perioden'],axis = 1) #they are all the same
rivm = rivm[rivm['WijkenEnBuurten'].str.contains('BU',case = False)] #we only use "buurten"
rivm = rivm.drop(['ID'],axis = 1)
rivm.columns = ['nbhood_code','moderate_high_risk','high_risk'] #only need these

I live in Den Bosch ('s-Hertogenbosch) and for sanity check I choose to display all the neighborhoods of DB because I know some of them, and also I'm curious. 

In [ ]:
db_code = GM_WK_BU.ID[GM_WK_BU.Title.str.contains('hertogenbo', case = False)].item().strip()
buurten_db = rivm[rivm.WijkenEnBuurten.str.contains(db_code[2:],case = False)]
buurten_db = buurten_db.merge(GM_WK_BU[['ID', 'Title']], left_on='WijkenEnBuurten', right_on='ID', how='left')
display(buurten_db)

Verified: these numbers correspond to the numbers here https://www.rivm.nl/media/smap/depressie.html?gemeente=%27s-Hertogenbosch, so it's all good. 

In [1]:
#to explore the distribution:
plt.hist(rivm.moderate_high_risk)
#plt.title('Distribution of moderate and high risk of depression and anxiety, % of the population')
plt.show()
plt.figure(figsize=(600/120,800/120))
plt.hist(rivm.high_risk, bins = 5)
#plt.title('Distribution of neighborhoods, high risk of depression and anxiety',
 #        fontweight = 'bold',
  #       fontsize = 14)
plt.xticks(fontsize=14)  # Set x-axis tick names
plt.yticks(fontsize=14)
plt.xlabel("High risk, % of the population", fontsize = 19.5)
plt.ylabel("N neighborhoods", fontsize = 19.5)
plt.tight_layout()
plt.savefig('hist.eps', bbox_inches='tight')
plt.show()

In [ ]:
#more exploration

n_buurten = rivm.shape[0]
print(n_buurten) #this is how many neighbourhoods are available
max_moderate, min_moderate, mean_moderate, sd_moderate = \
max(rivm.moderate_high_risk), min(rivm.moderate_high_risk), rivm.moderate_high_risk.mean(), rivm.moderate_high_risk.std()
max_high, min_high, mean_high, sd_high = \
max(rivm.high_risk), min(rivm.high_risk), rivm.high_risk.mean(), rivm.high_risk.std()

print("Moderate to high risk values: \nmax {:.2f}, \nmin {:.2f}, \
\nmean {:.2f}, \nSD {:.2f}".format(max_moderate, min_moderate, mean_moderate, sd_moderate))
print("\nHigh risk values: \nmax {:.2f}, \nmin {:.2f}, \
\nmean {:.2f}, \nSD {:.2f}".format(max_high, min_high, mean_high, sd_high))

print("\nSpecial neighborhoods:")
print("Maximum moderate+high risk: \
{}".format(GM_WK_BU.ShortTitle[GM_WK_BU.ID  == rivm.nbhood_code[rivm.moderate_high_risk == max_moderate].item()].item()))
print("Maximum high risk: \
{}".format(GM_WK_BU.ShortTitle[GM_WK_BU.ID  == rivm.nbhood_code[rivm.high_risk == max_high].item()].item()))
print("Minimum moderate+high risk: \
{}".format(GM_WK_BU.ShortTitle[GM_WK_BU.ID  == rivm.nbhood_code[rivm.moderate_high_risk == min_moderate].item()].item()))
#print("Minimum high risk: \
#{}".format(GM_WK_BU.ShortTitle[GM_WK_BU.ID  == rivm.nbhood_code[rivm.high_risk == min_high]]))

plt.figure(figsize=(600/120,800/120))
rivm.high_risk.plot.box()
plt.xticks(ticks=[], labels=[])  # Set x-axis tick names
plt.yticks(fontsize=14)

plt.xlabel("", fontsize = 20)
plt.ylabel("High risk, % of the population", fontsize = 20)
plt.tight_layout()
plt.savefig('article BeneLearn/box.eps', bbox_inches='tight')
plt.show()

In [ ]:
#create intervals to discretize the targets
range_high = max_high - min_high
print("range high =", range_high)
num_intervals = 5

#create intervals-boxes:
interval_size = range_high / num_intervals
intervals = [min_high + (i + 1) * interval_size
             for i in range(num_intervals)]
print(intervals)

In [ ]:
rivm = rivm.dropna(subset=['high_risk'])
rivm = rivm.dropna(subset=['moderate_high_risk'])

#create a new column representing the categorical levels based on the high_risk values
rivm['level_high'] = pd.cut(rivm['high_risk'], bins=[-float('inf')] + intervals, labels=False) + 1

#count occurrences of each level
level_counts = rivm['level_high'].value_counts().sort_index()
print(level_counts)

## Sample neighbourhoods per level for download

The notebook aims at 10 000 sampling points (so 10 000 SVI). There are 5 levels, so ideally it should sample 2 000 images per level to get a balanced dataset of the desired size. However, only the first two levels have >= 2 000 neighbourhoods to sample from. For these two levels the sampling part of the code would have to sample 1 image p/neighbourhood, while for the next levels - > 1, and the size of the sample per neighbourhood will be different for every level. So in this part we split the dataset per level in order to proceed with the sampling this way. 

Also in practice sampling 2000 images from 7 neighbourhoods of the last level results in a biased sample: the images are often from almost the same viewpoint. In order to avoid that, the notebook merges levels 4 and 5. 

In [ ]:
#separate into 4 dataframes
first = rivm[rivm.level_high == 1]
second = rivm[rivm.level_high == 2]
third = rivm[rivm.level_high == 3]
fourth = rivm[rivm['level_high'].isin([4, 5])]

#n_sample for the number of neighborhoods to be sampled per level
n_sample = 2500 
sampled_first = first.groupby('level_high').apply(lambda x: x.sample(n=n_sample, replace=False)).reset_index(drop=True)
display(sampled_first)

sampled_second = second.groupby('level_high').apply(lambda x: x.sample(n=n_sample, replace=False)).reset_index(drop=True)
display(sampled_second)

n_sample = 833
sampled_third = third.groupby('level_high').apply(lambda x: x.sample(n=n_sample, replace=False)).reset_index(drop=True)
display(sampled_third)

#and we take all the 100+ neighbourhoods from levels 4 and 5

# create nbhood codes to look for in the geopandas df later on
codes_first = sampled_first.nbhood_code
codes_second = sampled_second.nbhood_code
codes_third = sampled_third.nbhood_code
codes_fourth = fourth.nbhood_code

#now specify the level of risk in a discretized way
first['level_high'] = 1
second['level_high'] = 2
third['level_high'] = 3
fourth['level_high'] = 4

## Load and use the geographic data

We use the division into neighbourhoods published by the Dutch Central Statistics Bureau (https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/wijk-en-buurtkaart-2020).

In [ ]:
#replace 'something/buurt_2020_v3.shp' with the actual path to your .shp file
shapefile_path = 'something/buurt_2020_v3.shp'
gdf = gpd.read_file(shapefile_path)

# Print the first few rows to inspect the data
print(gdf.head())

# Plot the shapefile
gdf.plot()
plt.show()

In [ ]:
#we don't need 200 columns:
gdf = gdf[['BU_CODE', 'BU_NAAM', 'WK_CODE', 'GM_CODE', 'GM_NAAM', 'POSTCODE', 'geometry']]

#transform the coordinates to lat lon
#determine the current CRS
print("Original CRS:", gdf.crs)

# Convert the CRS from EPSG:28992 to WGS84 (EPSG:4326)
gdf_latlon = gdf.to_crs(epsg=4326)

Let's also import the shape of the NL itself, so that the sampled neighborhoods don't float in the abyss.

In [ ]:
nl = unary_union(gdf_latlon.geometry)

# Create a new GeoDataFrame with the super-polygon
gdf_nl = gpd.GeoDataFrame({'geometry': [nl]}, crs="EPSG:4326")

# Plot the country's border
gdf_nl.plot()
plt.show()

In [ ]:
gdf_first = gdf_latlon[gdf_latlon['BU_CODE'].isin(codes_first)]
gdf_second = gdf_latlon[gdf_latlon['BU_CODE'].isin(codes_second)]
gdf_third = gdf_latlon[gdf_latlon['BU_CODE'].isin(codes_third)]
gdf_fourth = gdf_latlon[gdf_latlon['BU_CODE'].isin(codes_fourth)]

In [ ]:
#plot the neighbourhoods you sampled

fig, ax = plt.subplots()
gdf_nl.plot(facecolor='none', edgecolor='gray', ax=ax)
first.plot(ax=ax, color='pink')
second.plot(ax=ax, color='green')
third.plot(ax=ax, color='yellow')
fourth.plot(ax=ax, color='blue')

first_patch = mpatches.Patch(color='pink', label='very low')
second_patch = mpatches.Patch(color='green', label='low')
third_patch = mpatches.Patch(color='yellow', label='moderate')
fourth_patch = mpatches.Patch(color='blue', label='high&very high')
plt.legend(handles=[first_patch, second_patch, third_patch, fourth_patch], 
           bbox_to_anchor=(1.05, 1), loc='upper left')

plt.title("Sampled neighborhoods")
plt.savefig("some_path.png", dpi=300)
plt.show()

## Sample coordinates with images available, from GSV

In [ ]:
def check_street_view_exists(latitude, longitude, api_key):
    url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={latitude},{longitude}&key={api_key}"
    response = requests.get(url)
    metadata = response.json()
    if metadata['status'] == 'OK' and metadata['copyright'] == '© Google':
        image_date = metadata['date']
        image_year, image_month = map(int, image_date.split('-'))
        if 5 <= image_month <= 9: #specify the months for sampling here
            return True
    return False

def generate_coordinates(polygon, num_points):
    min_x, min_y, max_x, max_y = polygon.bounds
    points = []
    while len(points) < num_points:
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(random_point):
            if check_street_view_exists(random_point.y, random_point.x, api_key):
                points.append(random_point)
    return points

Below are the same functions with statements for debug. It's handy to use these if there is a need to figure out where the sampling gets stuck. For example, there are very little or no SVI for a certain neighbourhood for the desired time period - these will help to detect that situation.

In [ ]:
def check_street_view_exists(latitude, longitude, api_key):
    url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={latitude},{longitude}&key={api_key}"
    response = requests.get(url)
    metadata = response.json()
    is_image = False
    is_date = None
    if metadata['status'] == 'OK' and metadata['copyright'] == '© Google':
        is_image = True
        image_date = metadata['date']
        image_year, image_month = map(int, image_date.split('-'))
        if image_month < 5 or image_month > 9: #to produce "green" data use this: 5 <= image_month <= 9
            is_date = True
            return [True, is_image, is_date]
        else:
            is_date = False
    return [False, is_image, is_date]

def generate_coordinates(polygon, num_points):
    min_x, min_y, max_x, max_y = polygon.bounds
    points = []
    log = pd.DataFrame(columns = ['for_point', 'n_requests', 'no_SVI', 'wrong_date','unk_date'])
    n_requests = 0
    no_SVI = 0
    wrong_date = 0
    unk_date = 0
    while len(points) < num_points:
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(random_point):
            check = check_street_view_exists(random_point.y, random_point.x, api_key)
            n_requests +=1
            if not check[0]:
                if not check[1]:
                    no_SVI +=1
                if check[2] is False:
                    wrong_date +=1
                if check[2] is None:
                    unk_date +=1
            if (no_SVI % 50 == 0 and no_SVI != 0) or (wrong_date % 50 == 0 and wrong_date != 0) or (unk_date % 50 == 0 and unk_date != 0):
                print('no_SVI:', no_SVI, 'wrong_date:', wrong_date, 'unk_date:', unk_date)

            '''
            #you can also uncomment this part and comment the previous if statement
            #this will stop the code from looking after there was no image available for 300 attempts,
            #and return whatever points were found for that neighbourhood.
            #use the appropriate code later to determine which neighbourhoods don't in fact have coordinates
            if wrong_date == 300 or no_SVI == 300:
                #print("300 is reached, switching to the next neighborhood.")
                return points 
            '''
            if check[0]:
                points.append(random_point)
                log.loc[len(points)] = [len(points)] + [n_requests] + [no_SVI] + [wrong_date] + [unk_date]
                n_requests = 0
                no_SVI = 0
                wrong_date = 0
                unk_date = 0
    display(log)
    return points

As sampling might get stuck due to the unavailability of the data, one could process the dataset in chunks, saving the intermediate process. The functions for that are below. 

In [ ]:
#functions for step-by-step coordinates saving:
def process_gdf_chunk(gdf_chunk, output_file,n_per_nbhood):
    gdf_chunk['rand_points'] = gdf_chunk.apply(lambda x: generate_coordinates(x['geometry'], n_per_nbhood), axis=1)
    gdf_chunk.to_pickle(output_file)
    
def process_gdf_in_chunks(gdf, chunk_size, output_prefix,n_p_nbhood):
    num_chunks = math.ceil(len(gdf) / chunk_size)
    print(num_chunks, chunk_size, len(gdf))
    
    for i in range(num_chunks):
        start = i * chunk_size
        print(start)
        end = min((i + 1) * chunk_size, len(gdf))
        print(end)
        gdf_chunk = gdf.iloc[start:end]
        print(len(gdf_chunk))
        output_file = f"{output_prefix}_chunk_{i}.pkl"
        process_gdf_chunk(gdf_chunk, output_file, n_p_nbhood)
        print(f"Processed chunk {i + 1}/{num_chunks}, saved to {output_file}")

chunk_size = 50 #or specify other size
output_prefix = "fourth_level" #set the right prefix
process_gdf_in_chunks(gdf_fourth, chunk_size, output_prefix, 20) #here we sample 20 points, adjust that per level

In [ ]:
#to restart later:
chunk_size = 250
start_chunk =  6# Set this to the first unprocessed chunk.
output_prefix = "_first_level"
num_chunks = math.ceil(len(gdf_first) / chunk_size)
for i in range(start_chunk, num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(gdf_first))
    gdf_chunk = gdf_first.iloc[start:end]
    output_file = f"{output_prefix}_chunk_{i}.pkl"
    process_gdf_chunk(gdf_chunk, output_file, 1)
    print(f"Processed chunk {i + 1}/{num_chunks}, saved to {output_file}")


At this point all the coordinates for randpoints are saved in the chunks. They are checked for the availability of SVI within the certain time frame.

In [ ]:
#to merge:
def merge_chunks_to_gdf(gdfs, num_chunks, output_prefix):
    for i in range(*num_chunks):
        chunk_name = f"{output_prefix}_chunk_{i}.pkl"
        chunk_data = pd.read_pickle(chunk_name)
        gdfs.append(chunk_data) 
    return gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

merged = merge_chunks_to_gdf([], (0,20), output_prefix) 

#check if everything went right after you maybe restarted from some chunk:
merged = merged.drop_duplicates(subset='BU_CODE', keep='first')
print(merged['BU_CODE'].nunique()) #should be the amount of neighbourhoods you sampled
display(merged)

In [ ]:
#augment it with depression and anxiety scores and levels
merged = merged.merge(rivm, left_on='BU_CODE', right_on='nbhood_code', how='left')
merged = merged.drop(['moderate_high_risk'],axis = 1)

#save, load, re-create:
merged.to_pickle("merged_with_randpoints.pkl")
df=pd.read_pickle('merged_with_randpoints.pkl')
gdf_df = gpd.GeoDataFrame(df, geometry='geometry')
display(gdf_df)

##### To load the saved geodataframes later:

df = pd.read_pickle('whatever_gdf_you_saved.pkl')

gdf_4_levels = gpd.GeoDataFrame(df, geometry='geometry')

There are different ways to deal with the non-existence of SVI for a certain neighbourhood for a certain timeframe. I've tried to augment functions with print statements (available above) to see when it can't find the coordinates and for what reason. For one neighbourhood for one of the datasets (I collected data separately for "greenery" and "no greenery" months) it couldn't find any images for 100 000 attempts. So then it was obvious I have to skip one of the neighbourhoods, and then I had to deal with it, because then there were only 6 neighbourhoods available in the "very high" category, so I ended up merging this category with the previous one. What I found more useful was letting it save whatever it managed to find after some threshold number of attempts, and then check with the code below, how many neighourhoods lack how many coordinates. This approach bears a risk of not finding something that is there, but in general this is faster, and also if the differences between categories are not very big after skipping some neighbourhoods, then one could just proceed with the nearly balanced dataset. 

In [ ]:
none_rows = merged[merged['rand_points'].isnull()]
display(none_rows)
none_rows_level_high_counts = none_rows['level_high'].value_counts()
print("\nCounts in none_rows DataFrame:")
print(none_rows_level_high_counts)

#drop the rows with None from merged
merged = merged[merged['rand_points'].notnull()]
print(len(merged))

In [ ]:
#sample more
selected_nbhoods = pd.DataFrame(columns=rivm.columns)

for lev, count in none_rows_level_high_counts.items():
    all_that_level = rivm[rivm.level_high == lev]
    added = 0
    while added < count:
        selected = all_that_level.iloc[randint(0,len(all_that_level)-1)]
        if not selected.nbhood_code in gdf_df.BU_CODE.values:
            selected_nbhoods = pd.concat([selected_nbhoods, selected.to_frame().T], ignore_index=True)
            added += 1

gdf_selected_nbhoods = gdf[gdf['BU_CODE'].isin(selected_nbhoods['nbhood_code'])]
gdf_selected_nbhoods = gdf_selected_nbhoods[['BU_CODE', 'BU_NAAM', 'WK_CODE', 'GM_CODE', 'GM_NAAM', 'POSTCODE', 'geometry']]
gdf_selected_nbhoods = gdf_selected_nbhoods.to_crs(epsg=4326)
#not sure for what I used the line below:
#gdf_selected_nbhoods = gdf_selected_nbhoods.merge(selected_nbhoods, left_on='BU_CODE', right_on='nbhood_code', how='left')

gdf_selected_nbhoods['rand_points'] = gdf_selected_nbhoods.apply(lambda x: generate_coordinates(x['geometry'], 40), axis=1)

#merge it with merged
gdf_selected_nbhoods = gdf_selected_nbhoods.drop(['moderate_high_risk'],axis = 1)
merged = pd.concat([merged, gdf_selected_nbhoods], ignore_index=True)


## Plot the locations of the sampled coordinates

In [ ]:
def plot_points_in_nbh(output_folder, gpd_df):
    for i in range(len(gpd_df)):
        # Get the geometry and rand_points for the selected index
        geometry = gpd_df.loc[i, 'geometry']
        rand_points = gpd_df.loc[i, 'rand_points']

        rand_points = gpd.GeoDataFrame({'geometry': rand_points})

        #Title for the plot
        high = gpd_df.high_risk.iloc[i]
        level = gpd_df.level_high.iloc[i]
        name_nbh = gpd_df.BU_NAAM.iloc[i]
        munic = gpd_df.GM_NAAM.iloc[i]
        t = "GSV locations for "+name_nbh+" in "+munic+", \nhigh_risk "+str(high)+", level "+str(level)

        #this is for y-axis tick labels formatting
        def format_decimal(x, pos=None):
            return f"{x:.3f}"

        fig, ax = plt.subplots()
        gpd_df.iloc[[i]].plot(ax=ax, color='#CCCCCC', edgecolor='black')
        rand_points.plot(ax=ax, color='red', markersize=5)
        plt.title(t)
        ax.yaxis.set_major_formatter(FuncFormatter(format_decimal))
       
        image_name = f"image_{i}.png"  # You can customize the naming
        image_path = os.path.join(output_folder, image_name)
        plt.savefig(image_path, dpi=300, bbox_inches='tight')

        plt.show()

#extract lat-lon coordinates
def points_to_lat_lon(points_list):
    return [(point.y, point.x) for point in points_list]

gdf_df['lat_lon_points'] = gdf_df['rand_points'].apply(points_to_lat_lon)
output_folder = 'some_path_for_plots' #write the path here!!
plot_points_in_nbh(output_folder, gdf_df[:5]) #plots points for the first 5 nbhoods

## Download the images, using the sampled coordinates

In [ ]:
all_data = pd.concat([first, second, third, fourth], ignore_index=True)
all_data.to_pickle("all.pkl")

In [ ]:
def download_streetview_image(latitude, longitude, api_key, save_path, file_name, size="512x512"):
    url = f"https://maps.googleapis.com/maps/api/streetview?size={size}&location={latitude},{longitude}&key={api_key}"
    
    response = requests.get(url, stream=True)
    
    if response.status_code == 200:
        with open(os.path.join(save_path, file_name), "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    else:
        print(f"Error: {response.status_code} - {response.text}")

save_path = "./nl_svi"  # Replace with your desired save path
targets = []
count = 0

for _, row in all_data.iterrows(): 
    risk = row.high_risk
    level_high = row.level_high
    nbhood = row.nbhood_code
    for p in row.rand_points:
        latitude = p.y
        longitude = p.x
        filename = str(count)+"_"+str(round(latitude,4))+"_"+str(round(longitude,4))+".jpg"
        ID = f"{count}_{latitude:.10f}_{longitude:.10f}"
        download_streetview_image(latitude, longitude, api_key, save_path, filename)
        targets.append({'ID':ID, 'file_name': filename, 'target': risk, 'discrete_target':level_high, 'nbhood':nbhood})
        count += 1

targets = pd.DataFrame(targets)
targets['discrete_target'] = targets['discrete_target'] - 1 #for classifier it's better this way
display(targets)
targets.to_csv('targets_and_coords.csv', index=False)

#print counts per level:
counts = targets['discrete_target'].value_counts()
print(counts)

# For continuous targets

### Loading data

CustomDataset class that is adjusted to load the images with continuous targets.

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, img_folder, transform=None):
        self.df = df
        self.img_folder = img_folder
        self.transform = transform
        self.min_target = self.df['high_risk'].min()
        self.max_target = self.df['high_risk'].max()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name = self.df.iloc[index]['file_name']
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        target = self.df.iloc[index, 4]
        #target_normalized = (target - self.min_target) / (self.max_target - self.min_target) #can also try this
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(target, dtype=torch.float) #torch.tensor(target_normalized, dtype=torch.float)


In [ ]:
# Define the data transforms necessary for the network that's being used
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
# Read the csv file
df = pd.read_csv('disp_coords.csv')

# Shuffle the DataFrame
df = df.sample(frac=1)

# Calculate sizes of each split
train_size = int(len(df) * 0.7) #set to less for hyperparameter selection
val_size = int(len(df) * 0.15)

# Split the data
train_df, val_df, test_df = np.split(df, [train_size, train_size + val_size])

# Save the indices
torch.save(train_df.index.values, 'train_indices_continuous_tiny.pt')
torch.save(val_df.index.values, 'val_indices_continuous_tiny.pt')
torch.save(test_df.index.values, 'test_indices_continuous_tiny.pt')

# Create datasets
train_dataset = CustomDataset(train_df, './nl_svi_dispersed_green', transform=train_transforms) #set this to train_transforms for random crop
val_dataset = CustomDataset(val_df, './nl_svi_dispersed_green', transform=val_transforms)
test_dataset = CustomDataset(test_df, './nl_svi_dispersed_green', transform=val_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False, num_workers=2)

### Function

In [ ]:
def train_custom_model(model_name, unfreeze_layers, optimizer_fn, lr, dropout_par, num_epochs, *args, **kwargs):
    model = torch.hub.load('facebookresearch/deit:main', model_name, pretrained=True)

    # Get the number of transformer layers
    num_transformer_layers = len(model.blocks)

    # Freeze all layers except the specified number of unfrozen layers
    for i in range(num_transformer_layers - unfreeze_layers):
        for param in model.blocks[i].parameters():
            param.requires_grad = False

    num_classes = 1
    in_features = model.head.in_features

    model.head = nn.Sequential(
        nn.Dropout(dropout_par),
        nn.BatchNorm1d(num_features=in_features),
        nn.ReLU(),
        nn.Dropout(dropout_par),
        nn.Linear(in_features, num_classes)
    )
    model = model.to(device)
    #print("Model built.")
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optimizer_fn(model.parameters(), lr=lr, *args, **kwargs)

    # Define the learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)

    # Early stopping parameters
    early_stopping_patience = 10
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    #custom metric to discretize the performance measurement
    how_close = 2.6 #it's SD for our data, you can try different thresholds

    for epoch in range(num_epochs):
        # Training
        train_loss = 0.0
        train_approx_right = 0

        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            # Calculate the custom metric
            close_enough = torch.abs(outputs - labels.float()) <= how_close
            train_approx_right += torch.sum(close_enough).item()

        # Calculate average loss, MAE and MSE over one epoch
        train_loss = train_loss / len(train_loader.dataset)
        train_approx_acc = train_approx_right / len(train_loader.dataset)

        # Validation
        val_loss = 0.0
        val_approx_right = 0

        model.eval()

        # We won't need gradients for validation, so save memory and computation
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs).squeeze()

                # Compute loss
                loss = criterion(outputs, labels.float())

                # Update loss and acc
                val_loss += loss.item() * inputs.size(0)
                close_enough = torch.abs(outputs - labels.float()) <= how_close
                val_approx_right += torch.sum(close_enough).item()

        # Calculate average loss and acc over validation dataset
        val_loss = val_loss / len(val_loader.dataset)
        val_approx_acc = val_approx_right / len(val_loader.dataset)

        # Learning rate scheduler step
        scheduler.step(val_loss)

        # Print validation statistics
        print(f'Epoch [{epoch+1}/{num_epochs}], \
        Approx Train Acc: {train_approx_acc:.4f}, Avg MSE Train: {train_loss:.4f}, \
        Approx Val Acc: {val_approx_acc:.4f}, Avg MSE Val: {val_loss:.4f}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= early_stopping_patience:
            print("Early stopping triggered, stopping at epoch", epoch)
            break
    #return model #uncomment for full fine-tuning


### Model selection

In [ ]:
m_names = ['deit_tiny_patch16_224', 'deit_small_patch16_224', 'deit_base_patch16_224']
dropouts = [0, 0.2, 0.5, 0.7]
to_unfreeze = [0,1,3,5]
lr_s = [0.001, 0.005, 0.01]
optimizers = [optim.Adam, optim.SGD, optim.Adagrad]

for m_n in m_names:
    for drop in dropouts:
        for to_unfr in to_unfreeze:
            for lr in lr_s:
                for opt in optimizers:
                    print(m_n, "dropout:", drop, "layers:", to_unfr, "lr =", lr, str(opt), end='\n\n')
                    train_custom_model(m_n, to_unfr, opt, lr, drop, 15)
                    print('')

### Train the model

Choose the model that shows the most progress, based on the previous section. Change the percenetage of the data used (in the Loading data section). Fill in the hyperparameters below, accounting for more data if necessary (increase dropout, L2 regularization,...).

In [ ]:
model = train_custom_model('deit_base_patch16_224', 5, optim.Adagrad, 0.001, 0.2, 100, weight_decay=3e-4)
torch.save(model.state_dict(), 'deit_continuous.pth')

# For discretized targets

### Loading data

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, img_folder, transform=None):
        self.df = df
        self.img_folder = img_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name = self.df.iloc[index]['file_name']
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        target = self.df.iloc[index]['discrete_target'] #use "discrete_target" for 4 bins data
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(target, dtype=torch.long)

In [ ]:
# Define the data transforms necessary for the network that's being used
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Read the csv file
df = pd.read_csv('green_disp_coords_orig_bins.csv')

# Shuffle the DataFrame
df = df.sample(frac=1)

# Calculate sizes of each split
train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)

# Split the data
train_df, val_df, test_df = np.split(df, [train_size, train_size + val_size])

# Save the indices
torch.save(train_df.index.values, 'train_indices_orig_bins.pt')
torch.save(val_df.index.values, 'val_indices_orig_bins.pt')
torch.save(test_df.index.values, 'test_indices_orig_bins.pt')

# Create datasets
train_dataset = CustomDataset(train_df, './nl_svi_dispersed_green', transform=train_transforms) #set this to train_transforms for random crop
val_dataset = CustomDataset(val_df, './nl_svi_dispersed_green', transform=val_transforms)
test_dataset = CustomDataset(test_df, './nl_svi_dispersed_green', transform=val_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False, num_workers=2)


## DeiT

### Function

In [ ]:
def train_custom_model(model_name, unfreeze_layers, optimizer_fn, lr, dropout_par, num_epochs, *args, **kwargs):
    model = torch.hub.load('facebookresearch/deit:main', model_name, pretrained=True)

    # Get the number of transformer layers
    num_transformer_layers = len(model.blocks)

    # Freeze all layers except the specified number of unfrozen layers
    for i in range(num_transformer_layers - unfreeze_layers):
        for param in model.blocks[i].parameters():
            param.requires_grad = False

    num_classes = 4
    in_features = model.head.in_features

    model.head = nn.Sequential(
        nn.Dropout(dropout_par),
        nn.BatchNorm1d(num_features=in_features),
        nn.ReLU(),
        nn.Dropout(dropout_par),
        nn.Linear(in_features, num_classes)
    )
    model = model.to(device)
    #print("Model built.")
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_fn(model.parameters(), lr=lr, *args, **kwargs)

    # Define the learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)

    # Early stopping parameters
    early_stopping_patience = 10
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        #print("Start epoch.")
        # Training
        train_loss, train_correct, train_total, train_f1_total = 0.0, 0, 0, 0
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            train_correct += torch.sum(preds == labels.data)
            train_total += labels.size(0)
            train_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)

        train_loss /= train_total
        train_acc = train_correct.item() / train_total
        train_f1 = train_f1_total / train_total

        # Validation
        val_loss, val_correct, val_total, val_f1_total = 0.0, 0, 0, 0
        model.eval()
        #print("Started validation.")
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(val_loader):
                #if i%10 == 0 and i !=0:
                 # print("Batch", i)
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                val_correct += torch.sum(preds == labels.data)
                val_total += labels.size(0)
                val_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)

        val_loss /= val_total
        val_acc = val_correct.item() / val_total
        val_f1 = val_f1_total / val_total

        scheduler.step(val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}],\nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= early_stopping_patience:
            print("Early stopping triggered, stopping at epoch", epoch)
            break
    return model #comment this line for hyperparameter selection


### Hyperparameter selection

Below is the code to perform grid search. The hyperparameters include three types of the DeiT models, amount of layers to re-train, learning rates and optimizers.

In [ ]:
m_names = ['deit_tiny_patch16_224', 'deit_small_patch16_224', 'deit_base_patch16_224']
to_unfreeze = [0,1,3,5]
lr_s = [0.001, 0.005, 0.01]
optimizers = [optim.Adam, optim.SGD, optim.Adagrad]

for m_n in m_names:
  for to_unfr in to_unfreeze:
    for lr in lr_s:
      for opt in optimizers:
        print(m_n, to_unfr, lr, str(opt), end='\n\n')
        train_custom_model(m_n, to_unfr, opt, lr, 0)
        print('')

### Train top-1 model with regularization and save weights

In [ ]:
model = train_custom_model('deit_base_patch16_224', 5, optim.SGD, 0.001, 0.2, 100, weight_decay=1e-4)
torch.save(model.state_dict(), 'base_5_SGD_001_reg.pth')

### For experiment with full landscapes

Code to conduct an experiment where the model is trained on the full landscapes, so the images are resized to the dimensions expected by the model and no random crop is used.

In [ ]:
#this assumes you have already run the code above and thus only have to re-define the train part

# Create datasets
train_dataset = CustomDataset(train_df, './nl_svi_dispersed_green', transform=val_transforms) #set this to val_transforms to avoid random crop

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

#### Grid search

In [ ]:
m_names = ['deit_tiny_patch16_224', 'deit_small_patch16_224', 'deit_base_patch16_224']
to_unfreeze = [0,1,3,5]
lr_s = [0.001, 0.01]
optimizers = [optim.Adam, optim.SGD, optim.RMSprop, optim.Adagrad]
dropouts = [0, 0.2, 0.4]
L2s = [0,1e-3]

for to_unfr in to_unfreeze:
  print("\n{} LAYERS UNFROZEN\n".format(to_unfr))
  for m_n in m_names:
    for lr in lr_s:
      for opt in optimizers:
        for do in dropouts:
          for w_d in L2s:
            print(m_n, to_unfr, str(opt), lr, do, w_d, end='\n\n')
            train_custom_model(m_n, to_unfr, opt, lr, do, 15, weight_decay=w_d)
            print('')

### To load later

In [ ]:
custom_dataset = CustomDataset(pd.read_csv('green_dispersed_coords.csv'), './nl_svi_dispersed_green', transform=val_transforms)

# Load the indices (when resuming)
#train_indices = torch.load('train_indices_final.pt')
#val_indices = torch.load('val_indices_final.pt')
test_indices = torch.load('test_indices_resnet.pt')

# Create the datasets using the loaded indices
#train_dataset = Subset(custom_dataset, train_indices)
#val_dataset = Subset(custom_dataset, val_indices)
test_dataset = Subset(custom_dataset, test_indices)

# Create data loaders
#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
#val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
#load the model
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=False)

# Get the number of transformer layers
num_transformer_layers = len(model.blocks)

# Freeze all layers except the specified number of unfrozen layers
for i in range(num_transformer_layers - 5):
    for param in model.blocks[i].parameters():
        param.requires_grad = False

num_classes = 4
in_features = model.head.in_features

model.head = nn.Sequential(
    nn.Dropout(),
    nn.BatchNorm1d(num_features=in_features),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(in_features, num_classes)
)
model = model.to(device)

model.load_state_dict(torch.load('deit_on_resnet_sets.pth')) #and the other one is 'deit_on_resnet_sets.pth'
model.eval()

### Evaluation and preprocessing for visualization

In [ ]:
criterion = nn.CrossEntropyLoss()
test_loss, test_correct, test_correct_adj, test_total, test_f1_total = 0.0, 0, 0, 0, 0

model.eval()
num_batches = len(test_loader)

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        test_correct += torch.sum(preds == labels.data)
        test_correct_adj += torch.sum(preds == labels.data)
        test_correct_adj += torch.sum( (preds +1) == labels.data) #to compute adjusted acc, comment this for standard acc
        test_correct_adj += torch.sum( (preds -1) == labels.data) #to compute adjusted acc, comment this for standard acc
        test_total += labels.size(0)
        test_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)

test_loss /= num_batches
test_acc = test_correct.item() / test_total
test_acc_adj = test_correct_adj.item() / test_total
test_f1 = test_f1_total / test_total

print('Test performance:')
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.5f}, Test Adj Acc: {test_acc_adj:.5f}, Test F1: {test_f1:.4f}')

In [ ]:
%matplotlib inline

def imshow(img):
    img = img * torch.tensor([0.229, 0.224, 0.225])[:, None, None] + torch.tensor([0.485, 0.456, 0.406])[:, None, None]  # Unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(30, 3))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

criterion = nn.CrossEntropyLoss()
test_loss, test_correct, test_total, test_f1_total = 0.0, 0, 0, 0
model.eval()
with torch.no_grad():
  for batch_index, (inputs, labels) in enumerate(test_loader):
    if batch_index == 0:  # Only take the first batch
        inputs, labels = inputs[10:20].to(device), labels[10:20].to(device)  # Select the first 10 examples
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        test_correct += torch.sum(preds == labels.data)
        test_total += labels.size(0)
        test_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)
        print('preds:', preds)
        print('labels:',labels.data)

        inputs_cpu = inputs.cpu()
        imshow(torchvision.utils.make_grid(inputs_cpu, nrow=10))
    else:
      break

test_acc = test_correct.item() / test_total
test_f1 = test_f1_total / test_total

print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}')


## ResNet50


### Function


In [ ]:
def train_custom_model_resnet50(unfreeze_layers, optimizer_fn, lr, **kwargs):
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1) # Load ResNet50 instead of DeiT

    # Get the number of layers in the ResNet50 model
    num_layers = len(list(model.children()))

    # Freeze all layers except the specified number of unfrozen layers
    for i, child in enumerate(model.children()):
        if i < num_layers - unfreeze_layers:
            for param in child.parameters():
                param.requires_grad = False

    num_classes = 4
    in_features = model.fc.in_features  # Get the number of input features for the fully connected layer

    model.fc = nn.Sequential(
        nn.Dropout(0.3),
        nn.BatchNorm1d(num_features=in_features),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(in_features, num_classes)
    )
    model = model.to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_fn(model.parameters(), lr=lr, **kwargs)

    # Define the learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)

    # Early stopping parameters
    early_stopping_patience = 10
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    n_batch_train = len(train_loader)
    n_batch_val = len(val_loader)

    num_epochs = 100
    for epoch in range(num_epochs):
        # Training
        train_loss, train_correct, train_total, train_f1_total = 0.0, 0, 0, 0
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            train_correct += torch.sum(preds == labels.data)
            train_total += labels.size(0)
            train_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)

        train_loss /= n_batch_train
        train_acc = train_correct.item() / train_total
        train_f1 = train_f1_total / train_total

        # Validation
        val_loss, val_correct, val_total, val_f1_total = 0.0, 0, 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                val_correct += torch.sum(preds == labels.data)
                val_total += labels.size(0)
                val_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)

        val_loss /= n_batch_val
        val_acc = val_correct.item() / val_total
        val_f1 = val_f1_total / val_total

        scheduler.step(val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}],\nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= early_stopping_patience:
            print("Early stopping triggered, stopping at epoch", epoch)
            break
    return model

### Model selection

In [ ]:
to_unfreeze = [0,1,3,5]
lr_s = [0.001, 0.005, 0.01]
optimizers = [optim.Adam, optim.SGD, optim.RMSprop, optim.Adagrad]

for to_unfr in to_unfreeze:
  for lr in lr_s:
    for opt in optimizers:
      print("Training ResNet50 with {} last layers unfrozen, lr {}, {}".format(to_unfr, lr, str(opt)), end='\n\n')
      train_custom_model_resnet50(to_unfr, opt, lr)
      print('')

### Actual model training

In [ ]:
model = train_custom_model_resnet50(3, optim.Adagrad,  0.001, weight_decay=1e-4)
torch.save(model.state_dict(), 'resnet_3_001_Adam_reg_deit_sets.pth')

### Load model later

In [ ]:
custom_dataset = CustomDataset(pd.read_csv('green_dispersed_coords.csv'), './nl_svi_dispersed_green', transform=val_transforms)

# Load the indices (when resuming)
#train_indices = torch.load('test_indices_bins1_resnet.pt')
val_indices = torch.load('val_indices_bins1_resnet.pt')
test_indices = torch.load('test_indices_resnet.pt')

# Create the datasets using the loaded indices
#train_dataset = Subset(custom_dataset, train_indices)
val_dataset = Subset(custom_dataset, val_indices)
test_dataset = Subset(custom_dataset, test_indices)

# Create data loaders
#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
unfreeze_layers = 3
num_classes = 4

#Load the model and adjust the architecture
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1) # Load ResNet50 instead of DeiT

num_layers = len(list(model.children())) # Get the number of layers in the ResNet50 model

for i, child in enumerate(model.children()): # Freeze all layers except the specified number of unfrozen layers
    if i < num_layers - unfreeze_layers:
        for param in child.parameters():
            param.requires_grad = False

in_features = model.fc.in_features  # Get the number of input features for the fully connected layer

model.fc = nn.Sequential(
    nn.Dropout(0),
    nn.BatchNorm1d(num_features=in_features),
    nn.ReLU(),
    nn.Dropout(0),
    nn.Linear(in_features, num_classes)
)

model = model.to(device)

#Load the weights
model.load_state_dict(torch.load("resnet_3_Adagrad_01_reg.pth"))


### Evaluation on the test set

We use the test set and the random prediction here for comparison, and we then visualize the images for all these predictions.

In [ ]:
criterion = nn.CrossEntropyLoss()
test_loss, test_correct, test_correct_adj, test_total, test_f1_total = 0.0, 0, 0, 0, 0
model.eval()
num_batches = len(test_loader)

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        test_correct += torch.sum(preds == labels.data)
        test_correct_adj += torch.sum(preds == labels.data)
        test_correct_adj += torch.sum( (preds +1) == labels.data) #to compute adjusted acc, comment this for standard acc
        test_correct_adj += torch.sum( (preds -1) == labels.data) #to compute adjusted acc, comment this for standard acc
        test_total += labels.size(0)
        test_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)

test_loss /= num_batches
test_acc = test_correct.item() / test_total
test_acc_adj = test_correct_adj.item() / test_total
test_f1 = test_f1_total / test_total

print('Test performance of ResNet50')
print(f'Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.5f}, Test Adj Acc: {test_acc_adj:.5f}, Test F1: {test_f1:.5f}')


In [ ]:
%matplotlib inline

def imshow(img):
    img = img * torch.tensor([0.229, 0.224, 0.225])[:, None, None] + torch.tensor([0.485, 0.456, 0.406])[:, None, None]  # Unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(30, 3))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

criterion = nn.CrossEntropyLoss()
test_loss, test_correct, test_total, test_f1_total = 0.0, 0, 0, 0
model.eval()
with torch.no_grad():
  for batch_index, (inputs, labels) in enumerate(test_loader):
    if batch_index == 0:  # Only take the first batch
        inputs, labels = inputs[:10].to(device), labels[:10].to(device)  # Select the first 10 examples
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        test_correct += torch.sum(preds == labels.data)
        test_total += labels.size(0)
        test_f1_total += f1_score(labels.cpu().detach(), preds.cpu().detach(), average='weighted') * labels.size(0)
        print('preds:', preds)
        print('labels:',labels.data)

        inputs_cpu = inputs.cpu()
        imshow(torchvision.utils.make_grid(inputs_cpu, nrow=10))
    else:
      break

test_loss /= test_total
test_acc = test_correct.item() / test_total
test_f1 = test_f1_total / test_total

print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}')


# Explaining


## Maybe useful plotting function

In [ ]:
for batch_index, (i, l) in enumerate(test_loader):
    if batch_index == 0:  # Only take the first batch
        inputs, labels = i[:10].to(device), l[:10].to(device)  # Select the first 10 examples
        print(len(inputs))
        inp = inputs[1]
        lab = labels[1]
    else:
      break

### To print one image or a grid

In [ ]:
#%matplotlib inline

def showGrid(img):
    img = img * torch.tensor([0.229, 0.224, 0.225])[:, None, None] + torch.tensor([0.485, 0.456, 0.406])[:, None, None]  # Unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(30, 3))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def showIm(img):
    img = img * torch.tensor([0.229, 0.224, 0.225])[:, None, None] + torch.tensor([0.485, 0.456, 0.406])[:, None, None]  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
print(labels)
inputs_cpu = inputs.cpu()
showGrid(torchvision.utils.make_grid(inputs_cpu, nrow=10))

In [ ]:
print("Label:", lab)
inp = inp.to('cpu')
showIm(inp)

## Getting the top predictions

In [ ]:
def get_extreme_indices():
    extreme_indices = []
    logits = []

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch_index, (inputs, labels) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            correct_predictions = (preds == labels.data)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

            for i, is_correct in enumerate(correct_predictions):
                if is_correct:
                    # Compute the index of the sample in the dataset
                    sample_index = batch_index * test_loader.batch_size + i
                    correct_class = labels[i].item()
                    # Store the index, correct class, and corresponding logit in a tuple
                    correct_logit = outputs[i, labels[i]].item()
                    extreme_indices.append((sample_index, correct_class, correct_logit))
                    # Store the logit corresponding to the correct prediction
                    logits.append(correct_logit)

    # Create the confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Sort the tuples based on the most extreme logits
    sorted_tuples = sorted(extreme_indices, key=lambda x: abs(x[2]), reverse=True)
    return sorted_tuples, cm

model.eval()
extreme_tuples, cm = get_extreme_indices()


In [ ]:
print(extreme_tuples)

In [ ]:
with open('extreme_deit_deit.pkl', 'wb') as f:
    pickle.dump(extreme_tuples, f)

In [ ]:
with open('extreme_resnet.pkl', 'rb') as f:
    extreme_tuples = pickle.load(f)


In [ ]:
def plot_confusion_matrix(cm, name):
    tick_names = ['very low', 'low', 'moderate', 'high&v. high']
    plt.figure(figsize=(10, 7))
    ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 16})  # Increase label size here
    plt.xticks(ticks=[0.5, 1.5, 2.5, 3.5], labels=tick_names, fontsize=14)  # Set x-axis tick names
    plt.yticks(ticks=[0.5, 1.5, 2.5, 3.5], labels=tick_names, fontsize=14)  # Set y-axis tick names
    plt.xlabel('Predicted', fontsize=20)  # Increase x-axis label size
    plt.ylabel('Truth', fontsize=20)  # Increase y-axis label size
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=14)
    plt.tight_layout()  # Remove margins
    plt.savefig(name + '.eps', bbox_inches='tight')  # Save with tight bounding box
    plt.show()

In [ ]:
plot_confusion_matrix(cm,'conf_deit')

In [ ]:
def get_top_images_per_category(indices, top_n=10):
    top_images = {i: [] for i in range(4)}
    for index, class_num, logit in indices:
        if len(top_images[class_num]) < top_n:
            top_images[class_num].append(index)
        if all(len(images) >= top_n for images in top_images.values()):
            break
    return top_images

top_n = 10
top_images_per_category = get_top_images_per_category(extreme_tuples, top_n)

In [ ]:
for class_num, image_indices in top_images_per_category.items():
    print(f"Top {top_n} images for class {class_num}:")
    for index in image_indices:
        image_t, target = test_dataset[index]
        showIm(image_t)
    print("\n")


## SHAP

https://github.com/shap/shap

In [ ]:
#just like with timm, you need to install this every time when you work in the cloud
!pip install shap

In [ ]:
import shap

In [ ]:
for class_num, image_indices in top_images_per_category.items():
  print(class_num)
  print(image_indices)
  print("")

In [ ]:
to_explain= []

for class_num, image_indices in top_images_per_category.items():
    for index in image_indices:
        image_t, target = test_dataset[index]
        to_explain.append(image_t)

to_explain = torch.stack(to_explain).cuda()

def normalize(tensor):
    tensor_min = tensor.min()
    tensor_max = tensor.max()
    return (tensor - tensor_min) / (tensor_max - tensor_min)


In [ ]:
class_names = ["Class 0", "Class 1", "Class 2", "Class 3"]
model.eval()

# Use GradientExplainer
background_indices = np.random.choice(len(test_dataset), 50, replace=False)
background = torch.stack([test_dataset[i][0] for i in background_indices]).cuda()
e = shap.GradientExplainer(model, background)

shap_values,indexes = e.shap_values(to_explain, ranked_outputs=4)


In [ ]:
# Get the names for the classes
index_names = np.vectorize(lambda x: class_names[x])(indexes.cpu())

#prepare the data to comply with the expectations of the plotting function
to_explain = to_explain.permute(0, 2, 3, 1)
shap_values = [np.transpose(sv, (0, 2, 3, 1)) for sv in shap_values]
to_explain_norm = normalize(to_explain)

# plot the explanations
shap.image_plot(shap_values, to_explain_norm.cpu().numpy(), index_names)

In [ ]:
# To save
torch.save(shap_values, 'shap_values_resnet_4.pt')
torch.save(indexes, 'indexes_resnet_4.pt')

In [ ]:
# To load
shap_values = torch.load('shap_values.pt')

## Gradient rollout

Gradient rollout was performed on the local CPU, not in the cloud. The code below would work on a PC, but not on cloud service.

**The code assumes you cloned Gildenblat's repository. Before running the code, go to ./vit-explain , inside the functions, and change the path to the model to the path where you saved the trained model, and re-create the model there if you only saved  weights (like this code generally does). There is code for re-creating the model in this notebook in sub-section DeiT/To load later. If you adjusted anything in the architecture of the model, it should also be adjusted there where it re-builds the model before it loads the weights.**

The application of Gradient Rollout to DeiT models is courtesy of J. Gildenblat. https://jacobgil.github.io/deeplearning/vision-transformer-explainability.

In [ ]:
os.chdir('./vit-explain')

In [ ]:
for class_num, dat in top_images_per_category.items():
    print(f"Top {top_n} images for class {class_num}")
    for index, logit in dat:
        original_index = test_indices[index]
        img_name = test_dataset.dataset.get_img_name(original_index)
        img_name = '../nl_svi_dispersed_green/'+img_name
        !python vit_explain.py --image_path {img_name} --head_fusion mean --discard_ratio 0.8 --logit {logit} --cl {class_num}
        !python vit_explain.py --image_path {img_name} --head_fusion max --discard_ratio 0.8 --logit {logit} --cl {class_num}
        !python vit_explain.py --image_path {img_name} --head_fusion min --discard_ratio 0.8 --logit {logit} --cl {class_num}
    print("\n")

### And with the least certain predictions:

In [ ]:
extreme_tuples.reverse()

In [ ]:
def get_bottom_images_per_category(indices, n=10):
    b_images = defaultdict(list)
    for index, class_num, logit in indices:
        if len(b_images[class_num]) < n:
            b_images[class_num].append((index, logit))

    return b_images

bottom_n = 10
b_images_per_category = get_bottom_images_per_category(extreme_tuples,bottom_n)

In [ ]:
for class_num, dat in b_images_per_category.items():
    print(f"Bottom {bottom_n} images for class {class_num}")
    for index, logit in dat:
        original_index = test_indices[index]
        img_name = test_dataset.dataset.get_img_name(original_index)
        img_name = '../nl_svi_green_10000/'+img_name
        !python vit_explain.py --image_path {img_name} --head_fusion mean --discard_ratio 0.8 --logit {logit} --cl {class_num}
        !python vit_explain.py --image_path {img_name} --head_fusion max --discard_ratio 0.8 --logit {logit} --cl {class_num}
        !python vit_explain.py --image_path {img_name} --head_fusion min --discard_ratio 0.8 --logit {logit} --cl {class_num}
    print("\n")